In [ ]:
import ads
import re
import os
from html import escape
from IPython.display import HTML, Latex

In [ ]:
ads_url = 'https://ui.adsabs.harvard.edu/abs/{paper.bibcode}'
doi_url = 'https://doi.org/{paper.doi[0]}'

# HTML
indent = ' ' * 10
nbsp = '&nbsp;'
nbhy = '&#8209;'
ampersand = '&amp;'
left_quote = '&ldquo;'
right_quote = '&rdquo;'
single_space = ' '
pre_item = indent + '  <li class="body">'
post_item = '</li>\n'
pre_list = indent + '<ol class="body publications" reversed>\n'
post_list = indent + '</ol>\n'
pre_title = indent + '<span class="head">'
post_title = '</span>\n'
pre_note = indent + '<span class="body">'
post_note = '</span>\n'
encode = escape
display = HTML
def link(url, text):
    return f"<a href='{url}'>{text}</a>"
output = 'cv.html'

# # LaTeX
# nbsp = '~'
# nbhy = '-'
# left_quote = '``'
# right_quote = "''"
# ampersand = '\\&'
# single_space = '\\ '
# pre_item = '    \\item '
# post_item = '\n'
# pre_list = '\\begin{etaremune}\n'
# post_list = '\\end{etaremune}\n'
# pre_title = '\\paragraph{'
# post_title = '}\n'
# pre_note = '\\paragraph{'
# post_note = '}\n'
# encode = lambda x: x.replace('&', ampersand)
# display = Latex
# def link(url, text):
#     return f'\\href{{{{{url}}}}}{{{{{text}}}}}'
# output = 'publications.tex'

In [ ]:
ads.config.token = os.environ['ADS_API_TOKEN']  # this is supposed to happen automatically but it doesn't
last_name = 'Hosseinzadeh'
query0 = 'author:"Hosseinzadeh, Griffin" (property:REFEREED OR bibstem:arXiv)'            # papers
query1 = 'author:"^Hosseinzadeh, G" bibstem:(ATel OR TNSCR OR GCN)'                      # first-author circulars
query2 = 'author:(Hosseinzadeh OR Hosselnzadeh) bibstem:(ATel OR TNSCR OR CBET OR GCN)'  # all circulars 

# completely ignore these
skip = [
    '2019arXiv190109962B',  # duplicate of 2019MNRAS.485.5120B
    '2018PhDT.......137H',  # thesis
    '2018PhDT.......176H',  # duplicate of thesis
    '2016arXiv160107368T',  # duplicate of 2016A&A...588A...5T
    '2016NatAs...1E..34L',  # corrigendum
]

# put these in the second section and list first n authors
emphasize = {
    '2022ApJ...937...13H': 2,
    '2022ApJ...931..159G': 2,
    '2021ApJ...921..180H': 2,
    '2020ApJ...905...94V': 2,
    '2019ApJ...884L..55G': 2,
    '2017Natur.551...64A': 2,
    '2017ApJ...848L..33A': 3,
    '2017ApJ...837L...2A': 2,
}

# use these as the author lists
override_authors = {
    '2017A&A...607A.115I': 'IceCube Collaboration et al.',
    '2017Natur.551...85A': 'LIGO/Virgo Collaboration et al.',
    '2017ApJ...848L..12A': 'LIGO/Virgo Collaboration et al.',
    '2018AJ....156..123A': 'Astropy Collaboration',
    '2022ApJ...935..167A': 'Astropy Collaboration',
}

stars = ['Hsu', 'Dauphin', 'Guevel']

In [ ]:
papers = ads.SearchQuery(q=query0, sort='date', rows=200,
                         fl=['author', 'first_author', 'year', 'title', 'pub', 'volume', 'page', 'doi', 'bibcode'])
papers1 = []
papers2 = []
papers3 = []
for paper in papers:
    if paper.bibcode in skip:
        continue
    elif paper.first_author.startswith(last_name):
        papers1.append(paper)
    elif paper.bibcode in emphasize:
        papers2.append(paper)
    else:
        papers3.append(paper)
print(len(papers1), len(papers2), len(papers3))

In [ ]:
circulars1 = list(ads.SearchQuery(q=query1, rows=300))
circulars2 = list(ads.SearchQuery(q=query2, rows=300))
print(len(circulars1), len(circulars2))

In [ ]:
sections = {'First Author': papers1, 'Major Contribution': papers2, 'Collaboration': papers3}
link1 = link(f"https://ui.adsabs.harvard.edu/search/q={query1}&sort=date desc", len(circulars1))
link2 = link(f"https://ui.adsabs.harvard.edu/search/q={query2}&sort=date desc", len(circulars2))
note = f'Plus {link2} astronomical telegrams and circulars ({link1} as first author).'

In [ ]:
def format_name(name):
    if 'Collaboration' in name:
        return name
    last_name, first_name = name.split(', ')
    given_names = re.split('[ -]+', first_name)
    space = nbhy if ('-' in first_name) else nbsp
    initials = [encode(n[0]) + '.' for n in given_names]
    all_initials = space.join(initials)
    encoded_name = all_initials + nbsp + encode(last_name)
    if last_name in stars:
        encoded_name += '*'
    return encoded_name


def format_names(names, n=2):
    if n is None:
        n = len(names)
    elif n >= 2:
        n = min(n, len(names))
    else:
        raise ValueError('must allow at least two authors (including et al.)')
    sep = ', ' if n > 2 else ' '
    formatted_names = [format_name(name) for name in names[slice(None, n)]]
    if n == len(names) > 1:
        formatted_names[-1] = ampersand + ' ' + formatted_names[-1]
    elif n > 1:
        formatted_names[-1] = 'et al.'
    return sep.join(formatted_names)
        

def format_journal(paper):
    bibstem = paper.bibcode[4:9].strip('.')
    if paper.page is not None and paper.page[0].startswith('L'):
        bibstem += 'L'
    return encode(bibstem)


def format_paper(paper, n=2):
    if paper.bibcode in override_authors:
        name_part = override_authors[paper.bibcode] + single_space
    else:
        name_part = format_names(paper.author, n=n) + single_space
    
    year_part = paper.year + ', '

    title_part = left_quote + paper.title[0].rstrip('.')
    if title_part[-1] not in '!?':
        title_part += ','
    title_part += right_quote + single_space

    if 'arXiv' in paper.bibcode:
        journal_part = ''
        volume_page_part = link(ads_url, paper.page[0])
    else:
        journal_part = link(doi_url, format_journal(paper)) + ', '
        if paper.page is None:
            volume_page_part = link(ads_url, 'in press')
        else:
            volume_page_part = link(ads_url, paper.volume + ', ' + paper.page[0])
    
    item = pre_item + name_part + year_part + title_part + journal_part + volume_page_part + post_item
    return item.format(paper=paper)


def format_papers(papers):
    formatted_papers = pre_list
    for paper in papers:
        if paper.bibcode in emphasize:
            n = emphasize[paper.bibcode] + 1
        else:
            n = 2
        formatted_papers += format_paper(paper, n=n)
    formatted_papers += post_list
    return formatted_papers


def format_sections(sections):
    formatted_sections = ''
    for title, sublist in sections.items():
        formatted_sections += pre_title + title + post_title
        formatted_sections += format_papers(sublist)
    formatted_sections += pre_note + note.format() + post_note
    return formatted_sections

In [ ]:
publication_list = format_sections(sections)
display(publication_list)

In [ ]:
with open(output) as f:
    cv = f.read()
pattern0 = re.escape(publication_list[:40])
pattern1 = re.escape(publication_list[-25:])
i0 = re.search(pattern0, cv).start()
i1 = re.search(pattern1, cv).end()
with open(output, 'w') as f:
    f.write(cv[:i0])
    f.write(publication_list)
    f.write(cv[i1:])